### 1.深度学习基础

### 线性回归

* 解析解(analytical solution)  
误差最小化问题的解可以直接用公式表达出来
* 数值解(numerical solution)  
大多数深度学习模型并没有解析解，只能通过优化算法有限次迭代模型参数来尽可能降低损失函数的值。
* 小批量随机梯度下降（mini-batch stochastic gradient descent）

In [5]:
# len()返回的是第一个维度的值
import torch
x = torch.tensor([[-1.4239, -1.3788],
        [ 0.0275,  1.3550],
        [ 0.7616, -1.1384],
        [ 0.2967, -0.1162],
        [ 0.0822,  2.0826],
        [-0.6343, -0.7222],
        [ 0.4282,  0.0235],
        [ 1.4056,  0.3506],
        [-0.6496, -0.5202],
        [-0.3969, -0.9951]])
print(len(x))

10


In [8]:
# 矩阵乘法
a = torch.randn(2, 3)
b = torch.randn(3, 2)
print(torch.mm(a, b))
print(torch.matmul(a, b))

tensor([[ 0.7779,  1.7195],
        [-1.1831, -1.3236]])
tensor([[ 0.7779,  1.7195],
        [-1.1831, -1.3236]])


### 小批量随机梯度下降优化算法
使用自动求梯度模块计算得到一个小批量的梯度和，再将其除以批量大小得到平均值

### 训练模型
* 读取小批量数据样本(特征和标签)
* 调用反向函数``backward``计算小批量梯度
* 调用优化算法迭代模型参数
(由于损失``l``不是一个标量,再次运行backward时需要先调用``.sum()``将损失求和变成一个标量)
* 每次更新完参数需要将参数的梯度清零

In [ ]:
# 训练部分代码

for epoch in range(num_epochs):  # 训练模型一共需要num_epochs个迭代周期
    # 在每一个迭代周期中，会使用训练数据集中所有样本一次（假设样本数能够被批量大小整除）。X
    # 和y分别是小批量样本的特征和标签
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y).sum()  # l是有关小批量X和y的损失
        l.backward()  # 小批量的损失对模型参数求梯度
        sgd([w, b], lr, batch_size)  # 使用小批量随机梯度下降迭代模型参数

        # 不要忘了梯度清零
        w.grad.data.zero_()
        b.grad.data.zero_()
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().item()))

### Pytorch的简洁实现

#### 使用``data``包实现数据集的组合和读取

In [ ]:
# 读取数据
使用PyTorch的data包来实现特征和标签的组合``data.TensorDataset``,数据的读取``data.DataLoader``

import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

### 定义模型
* ``torch.nn``即nerual network.
nn的核心数据结构是Module,常见做法为继承``nn.module``,撰写自己的额网络层.

* #### 使用``nn.module``实现一个线性回归模型


In [ ]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构


* #### 使用``nn.Sequential``更加方便的搭建网络  
Sequential是一个有序的容器,网络层按照传入容器中的顺序添加到计算图中.

* #### 通过``net.parameters()``查看模型的所有可学习参数  
此函数返回一个生成器  
for param in net.parameters():  
print(param)


* #### 通过``init.normal_()``将权重参数每个元素初始化为随机采样于均值为0、标准差为0.01的正态分布。偏差会初始化为零。

In [ ]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)


### 定义优化算法

* #### ``torch.optim``模块提供了很多常用的优化算法


* 优化器实例，并指定学习率为0.03的小批量随机梯度下降（SGD）为优化算法。  
import torch.optim as optim  
optimizer = optim.SGD(net.parameters(), lr=0.03)  
print(optimizer)

* 为不同子网络设置不同的学习率,如果对某个参数不指定学习率，就使用最外层的默认学习率  
optimizer =optim.SGD([   
      {'params': net.subnet1.parameters()}, # lr=0.03  
{'params': net.subnet2.parameters(), 'lr': 0.01}  
], lr=0.03)



* #### 注意,当需要修改学习率的时候

主要有两种做法。一种是修改optimizer.param_groups中对应的学习率，另一种是更简单也是较为推荐的做法——新建优化器，由于optimizer十分轻量级，构建开销很小，故而可以构建新的optimizer。但是后者对于使用动量的优化器（如Adam），会丢失动量等状态信息，可能会造成损失函数的收敛出现震荡等情况。

In [ ]:
# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1 # 学习率为之前的0.1倍


### 训练模型
* 调用``optim``实例的``step``函数来迭代模型参数.

In [ ]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

### softmax回归
对所有的分类类别输出做置信度估计,对每个类别的置信度取指数并统计每个类别的预测概率.softmax运算不改变预测类别输出.
* 解决了两个问题  
    * 将输出值变换成值为正且和为1的概率分布.   
    * 输出值范围限定后,更易于衡量真实标签和确定范围的输出值之间的误差.  


### 交叉熵损失函数
* 为什么不选用平方损失?  
平方损失过于严格.对于预测结果均正确的两个概率分布.正确标签为3.第一个预测概率输出为\[0.2,0.2,0.6\],第二个为\[0,0.4,0.6\].两者都有同样正确的分类预测结果,但是前者比后者的损失要小很多.过于严格,影响训练.

* 如何改善上述问题?  
使用更适合衡量两个概率分布差异的测量函数,其中,交叉熵（cross entropy）是一个常用的衡量方法.
交叉熵只关心对正确类别的预测概率，因为只要其值足够大，就可以确保分类结果正确。



### torchvision包
它是服务于PyTorch深度学习框架的，主要用来构建计算机视觉模型.  
主要由以下几部分构成:  

``torchvision.datasets``:一些加载数据的函数及常用的数据集接口；  
``torchvision.models``:包含常用的模型结构（含预训练模型），例如AlexNet、VGG、ResNet等；   
``torchvision.transforms``:常用的图片变换，例如裁剪、旋转等；  
``torchvision.utils``:含两个函数,一个是make_grid,它能将多张图片拼接在一个网格中;另一个是save_img,它能将tensor保存成图片. 


* 使用torchvision.datasets来下载数据集.  
* 参数transforms.ToTensor()  
    * 将尺寸为(H,W,C)且数据位于\[0, 255 \]的PIL图片 或 数据类型为np.uint8的numpy数组----> 尺寸为(C,H,W),数据类型为torch.float32且位于\[\0.0, 1.0]的Tensor.

In [ ]:
# torchcvision.datasets提供常用的数据集加载
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True, download=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False, download=True, transform=transforms.ToTensor())


### torch.utils.data.DataLoader
因为minst_train是``torch.utils.data.Dataset``的子类,所以可以将其传入``torch.utils.data.DataLoader``来创建数据样本的DataLoader实例.  
PyTorch的``DataLoader``一个很方便的功能是允许使用多进程来加速数据的读取.

In [ ]:
# 使用DataLoader读取小批量数据

batch_size = 256
if sys.platform.startswith('win'):
    num_workers = 0  # 0表示不用额外的进程来加速读取数据
else:
    num_workers = 4
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)


#### 对于要忽略,不进行使用的变量,可以用下划线_进行接收.
如: _, figs = plt.subplots(1, len(images), figsize=(12, 12))

### 定义一个在一行里画多张图像和对应标签的函数

In [ ]:

# 将数值标签转换成对应的文本文件
def get_fashion_mnist_labels(labels):
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]

# 在一行画多张图
def show_fashion_mnist(images, labels):
    d2l.use_svg_display()
    # 这里的_表示我们忽略（不使用）的变量
    _, figs = plt.subplots(1, len(images), figsize=(12, 12))
    for f, img, lbl in zip(figs, images, labels):
        f.imshow(img.view((28, 28)).numpy())
        f.set_title(lbl)
        f.axes.get_xaxis().set_visible(False)
        f.axes.get_yaxis().set_visible(False)
    plt.show()

# 看一下训练数据集中前10个样本的图像文件和文本标签.
X, y = [], []
for i in range(10):
    X.append(mnist_train[i][0])
    y.append(mnist_train[i][1])
show_fashion_mnist(X, get_fashion_mnist_labels(y))


### 多层感知机(multilayer perceptron, MLP)